In [1]:
# ==============================================================================
# KROK 1: GLOBÁLNÍ SETUP A IMPORTY
# ==============================================================================
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from copy import deepcopy
from math import pi
import random

# --- Konfigurace cesty k projektu (není potřeba měnit) ---
notebook_path = os.getcwd()
project_root = os.path.dirname(notebook_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- Import vašich vlastních modulů ---
import Systems
import Filters
from utils import losses, trainer, utils
from state_NN_models import StateBayesianKalmanNet, StateKalmanNet, StateKalmanNetWithKnownR

# --- Globální nastavení ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

# Nastavení seedu pro globální reprodukovatelnost
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

Používané zařízení: cuda


In [2]:
# ==============================================================================
# KROK 2: DEFINICE JEDNOTLIVÝCH SYSTÉMŮ
# ==============================================================================

def define_system_1d_nonlinear():
    print("\n--- Definuji Systém 1: 1D Nelineární (kubický) ---")
    state_dim, obs_dim = 1, 1
    
    # Skutečná dynamika
    f_true = lambda x: 0.9 * x - 0.05 * x**3 
    h_true = lambda x: 0.5 * x
    Q_true = torch.tensor([[0.1]])
    R_true = torch.tensor([[0.1]])
    Ex0_true = torch.tensor([[1.0]])
    P0_true = torch.tensor([[0.5]])
    sys_true = Systems.DynamicSystem(state_dim, obs_dim, f=f_true, h=h_true, Q=Q_true, R=R_true, Ex0=Ex0_true, P0=P0_true, device=device)

    # Nepřesný model
    f_model = lambda x: 0.9 * x 
    Q_model = torch.tensor([[0.01]])
    R_model = torch.tensor([[0.2]])
    Ex0_model = torch.tensor([[0.5]])
    P0_model = torch.tensor([[0.5]])
    sys_model = Systems.DynamicSystem(state_dim, obs_dim, f=f_model, h=h_true, Q=Q_model, R=R_model, Ex0=Ex0_model, P0=P0_model, device=device)
    
    return sys_true, sys_model

def define_system_2d_synthetic():
    print("\n--- Definuji Systém 2: 2D Syntetický (sinusový) ---")
    state_dim, obs_dim = 2, 2

    # Skutečná dynamika
    f_true = lambda x: 0.9 * torch.sin(1.1 * x + 0.1 * pi) + 0.01
    h_true = lambda x: 1.0 * (1.0 * x + 0.0)**2
    Q_true = torch.eye(state_dim) * 0.5
    R_true = torch.eye(obs_dim) * 0.1
    Ex0_true = torch.tensor([[1.0], [0.0]])
    P0_true = torch.eye(state_dim) * 1.5
    sys_true = Systems.DynamicSystem(state_dim, obs_dim, f=f_true, h=h_true, Q=Q_true, R=R_true, Ex0=Ex0_true, P0=P0_true, device=device)

    # Nepřesný model
    f_model = lambda x: 1.0 * torch.sin(1.0 * x + 0.0) + 0.0
    Q_model = torch.eye(state_dim) * 0.1
    R_model = R_true
    Ex0_model = torch.tensor([[0.5], [0.5]])
    P0_model = torch.eye(state_dim) * 1.0
    sys_model = Systems.DynamicSystem(state_dim, obs_dim, f=f_model, h=h_true, Q=Q_model, R=R_model, Ex0=Ex0_model, P0=P0_model, device=device)

    return sys_true, sys_model

def define_system_2d_linear():
    state_dim_2d = 2
    obs_dim_2d = 2

    F_base_2d = torch.tensor([[1.0, 1.0], 
                            [0.0, 1.0]])

    svd_F = torch.linalg.svd(F_base_2d)
    F_true_2d = F_base_2d / svd_F.S[0]

    H_true_2d = torch.eye(obs_dim_2d)

    Q_true_2d = torch.eye(state_dim_2d) * 0.5 # Šum procesu
    R_true_2d = torch.eye(obs_dim_2d) * 0.1 # Šum měření

    # Počáteční podmínky
    Ex0_true_2d = torch.tensor([[1.0], [0.0]])
    P0_true_2d = torch.eye(state_dim_2d) * 1.5
    F_model_2d = F_true_2d
    H_model_2d = H_true_2d
    Q_model_2d = torch.eye(state_dim_2d) * 0.1
    R_model_2d = R_true_2d
    Ex0_model_2d = torch.tensor([[0.5], [0.5]])
    P0_model_2d = torch.eye(state_dim_2d) * 1.0

    print("\nInicializuji 2D Linear_Canonical systém (replikace autorů)...")
    sys_true = Systems.DynamicSystem(
        state_dim=state_dim_2d, obs_dim=obs_dim_2d,
        Ex0=Ex0_true_2d, P0=P0_true_2d,
        Q=Q_true_2d, R=R_true_2d,
        F=F_true_2d, H=H_true_2d,
        device=device
    )
    sys_model = Systems.DynamicSystem(
        state_dim=state_dim_2d, obs_dim=obs_dim_2d,
        Ex0=Ex0_model_2d, P0=P0_model_2d,
        Q=Q_model_2d, R=R_model_2d,
        F=F_model_2d, H=H_model_2d,
        device=device
    )
    return sys_true, sys_model

In [3]:
# ==============================================================================
# KROK 3: DEFINICE "HELPER" FUNKCÍ PRO TRÉNINK A EVALUACI
# ==============================================================================

def run_training_for_system(train_loader, val_loader, sys_model, device):
    """
    Provede kompletní trénink všech tří NN modelů pro daný systém.
    Resetuje seed před každým tréninkem pro férové srovnání.
    Vrací slovník s natrénovanými modely.
    """
    print("\n" + "="*80)
    print("ZAČÍNÁM TRÉNINKOVOU FÁZI PRO AKTUÁLNÍ SYSTÉM")
    print("="*80)

    # --- Konfigurace ---
    # Konfigurace pro Bayesovský model
    bkn_model_config = {"hidden_size_multiplier": 12, "output_layer_multiplier": 4, "num_gru_layers": 1, "init_min_dropout": 0.5, "init_max_dropout": 0.8}
    bkn_train_config = {"total_train_iter": 1500, "learning_rate": 1e-4, "clip_grad": 1.0, "J_samples": 20, "validation_period": 20, "logging_period": 20, "warmup_iterations": 100}
    
    # Konfigurace pro klasické KalmanNety
    classic_model_config = {"hidden_size_multiplier": 12}
    classic_train_config = {"epochs": 150, "lr": 1e-4, "early_stopping_patience": 30, "clip_grad": 10.0}
    

    # --- 3. Trénink KalmanNet with Known R ---
    print("\n" + "-"*40)
    print("--- Trénuji KalmanNet with Known R (KNetR) ---")
    print("-" * 40)
    torch.manual_seed(42); np.random.seed(42); random.seed(42)
    model_knetR = StateKalmanNetWithKnownR(sys_model, device, **classic_model_config).to(device)
    trained_model_knetR = trainer.train_state_KalmanNet(model=model_knetR, train_loader=train_loader, val_loader=val_loader, device=device, **classic_train_config)
    print("--- Trénink KNetR dokončen. ---")
    
    # --- 1. Trénink Bayesian KalmanNet ---
    print("\n" + "-"*40)
    print("--- Trénuji Bayesian KalmanNet (BKN) ---")
    print("-" * 40)
    torch.manual_seed(42); np.random.seed(42); random.seed(42)
    model_bkn = StateBayesianKalmanNet(sys_model, device, **bkn_model_config).to(device)
    # Předpokládá se, že trénovací funkce je v utils.trainer
    results_bkn = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=model_bkn, train_loader=train_loader, val_loader=val_loader, device=device, **bkn_train_config)
    trained_model_bkn = results_bkn['final_model']
    print("--- Trénink BKN dokončen. ---")

    # --- 2. Trénink klasického KalmanNet (pouze MSE) ---
    # print("\n" + "-"*40)
    # print("--- Trénuji klasický KalmanNet (KNet) ---")
    # print("-" * 40)
    # torch.manual_seed(42); np.random.seed(42); random.seed(42)
    # model_classic = StateKalmanNet(sys_model, device, **classic_model_config).to(device)
    # trained_model_classic = trainer.train_state_KalmanNet(model=model_classic, train_loader=train_loader, val_loader=val_loader, device=device, **classic_train_config)
    # print("--- Trénink KNet dokončen. ---")
    
    return {
        'bkn': trained_model_bkn,
        'knetR': trained_model_knetR
    }


def run_evaluation_for_system(trained_models, test_loader, sys_true, sys_model, device):
    """
    Provede kompletní vyhodnocení všech modelů na testovacích datech.
    Vytiskne detailní logy pro každou trajektorii a finální přehlednou tabulku.
    """
    print("\n" + "="*80)
    print("ZAČÍNÁM EVALUAČNÍ FÁZI PRO AKTUÁLNÍ SYSTÉM")
    print("="*80)
    
    # --- Načtení natrénovaných modelů ---
    trained_model_bkn = trained_models['bkn']
    # trained_model_classic = trained_models['classic_knet']
    trained_model_knetR = trained_models['knetR']
    
    # --- Konfigurace testu ---
    NUM_TEST_TRAJ = len(test_loader.dataset)
    TEST_SEQ_LEN = test_loader.dataset.tensors[0].shape[1]
    J_SAMPLES_TEST = 25
    
    # --- Inicializace filtrů pro porovnání ---
    ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
    ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
    ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
    ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
    print("Model-based filtry (EKF, UKF) inicializovány.")

    # --- Vyhodnocovací smyčka ---
    all_x_true_cpu = []
    all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
    # all_x_hat_classic_knet_cpu = []
    all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
    all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
    all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
    all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
    all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []

    print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

    trained_model_bkn.eval(); trained_model_knetR.eval()

    with torch.no_grad():
        for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
            y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
            x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
            initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
            
            # C. KNet with Known R
            trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
            knetR_preds_x, knetR_preds_P = [], []
            for t in range(1, TEST_SEQ_LEN):
                x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
                knetR_preds_x.append(x_filtered_t)
                knetR_preds_P.append(P_filtered_t)
            full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
            full_P_hat_knetR = torch.cat([sys_model.P0.unsqueeze(0), torch.stack(knetR_preds_P, dim=0).squeeze(1)], dim=0)

            # A. Bayesian KalmanNet
            ensemble_trajectories = []
            for j in range(J_SAMPLES_TEST):
                trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
                current_x_hats = []
                for t in range(1, TEST_SEQ_LEN):
                    x_filtered_t, _ = trained_model_bkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
                    current_x_hats.append(x_filtered_t)
                ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
            ensemble = torch.stack(ensemble_trajectories, dim=0)
            predictions_bkn = ensemble.mean(dim=0)
            diff = ensemble - predictions_bkn.unsqueeze(0)
            covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
            full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
            full_P_hat_bkn = torch.cat([sys_model.P0.unsqueeze(0), covariances_bkn], dim=0)

            # B. Klasický KNet
            # trained_model_classic.reset(batch_size=1, initial_state=initial_state)
            # classic_knet_preds = []
            # for t in range(1, TEST_SEQ_LEN):
            #     x_filtered_t = trained_model_classic.step(y_test_seq_gpu[t, :].unsqueeze(0))
            #     classic_knet_preds.append(x_filtered_t)
            # full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
            
            # D. EKF
            ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
            full_x_hat_ekf_m = torch.cat([sys_model.Ex0.reshape(1, -1), ekf_m_res['x_filtered']], dim=0)
            full_P_hat_ekf_m = torch.cat([sys_model.P0.unsqueeze(0), ekf_m_res['P_filtered']], dim=0)
            
            ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
            full_x_hat_ekf_i = torch.cat([sys_true.Ex0.reshape(1, -1), ekf_i_res['x_filtered']], dim=0)
            full_P_hat_ekf_i = torch.cat([sys_true.P0.unsqueeze(0), ekf_i_res['P_filtered']], dim=0)

            # E. UKF
            ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
            full_x_hat_ukf_m = torch.cat([sys_model.Ex0.reshape(1, -1), ukf_m_res['x_filtered']], dim=0)
            full_P_hat_ukf_m = torch.cat([sys_model.P0.unsqueeze(0), ukf_m_res['P_filtered']], dim=0)
            
            ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
            full_x_hat_ukf_i = torch.cat([sys_true.Ex0.reshape(1, -1), ukf_i_res['x_filtered']], dim=0)
            full_P_hat_ukf_i = torch.cat([sys_true.P0.unsqueeze(0), ukf_i_res['P_filtered']], dim=0)
            
            # F. Uložení výsledků
            all_x_true_cpu.append(x_true_seq_gpu.cpu())
            all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
            # all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
            all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
            all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
            all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
            all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
            all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())

            print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

    # --- Finální výpočet a výpis metrik ---
    mse_bkn, anees_bkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
    mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
    mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
    
    print("\nPočítám finální metriky pro jednotlivé trajektorie...")
    with torch.no_grad():
        for i in range(NUM_TEST_TRAJ):
            x_true = all_x_true_cpu[i]
            def get_metrics(x_hat, P_hat):
                mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
                anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
                return mse, anees

            mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
            # mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
            mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
            mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
            mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
            mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
            mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)

    def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
    state_dim_for_nees = all_x_true_cpu[0].shape[1]

    print("\n" + "="*80)
    print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
    print("="*80)
    print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
    print("-" * 80)
    print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
    print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
    # print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
    print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
    print("-" * 80)
    print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
    print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
    print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
    print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
    print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
    print("="*80)

In [4]:
# ==============================================================================
# KROK 4a: KONFIGURACE A VÝBĚR EXPERIMENTU
# ==============================================================================

# --- Globální konfigurace pro generování dat ---
# Tato nastavení budou použita pro experiment spuštěný v další buňce.
DATA_CONFIG = {
    "TRAIN_SEQ_LEN": 10,
    "VALID_SEQ_LEN": 20,
    "TEST_SEQ_LEN": 300,
    "NUM_TRAIN_TRAJ": 500,
    "NUM_VALID_TRAJ": 200,
    "NUM_TEST_TRAJ": 40,
    "BATCH_SIZE": 8
}

# --- Slovník se všemi dostupnými systémy ---
# Klíč je název pro výpis, hodnota je funkce, která systém vytvoří.
systems_to_test = {
    "1D Nelineární (kubický)": define_system_1d_nonlinear,
    "2D Syntetický (sinusový)": define_system_2d_synthetic,
    "2D Lineární (easy)": define_system_2d_linear,
    # "Můj Třetí Systém": define_system_3_placeholder, # Zde můžete přidat další
}

# ##############################################################################
# ### ZDE SI VYBERTE, KTERÝ SYSTÉM CHCETE SPUSTIT ###
# ##############################################################################
# Jednoduše nastavte proměnnou `system_name_to_run` na klíč ze slovníku výše.

system_name_to_run = "1D Nelineární (kubický)"
# system_name_to_run = "2D Syntetický (sinusový)"
# system_name_to_run = "2D Lineární (easy)"

# ------------------------------------------------------------------------------
# Níže již nemusíte nic měnit. Další buňka si tuto proměnnou přečte a provede
# kompletní trénink a evaluaci pro vybraný systém.
# ------------------------------------------------------------------------------

# Získání správné funkce pro definici systému
try:
    define_system_func = systems_to_test[system_name_to_run]
    print(f"Připraven ke spuštění experimentu pro systém: '{system_name_to_run}'")
except KeyError:
    print(f"CHYBA: Systém s názvem '{system_name_to_run}' nebyl nalezen v `systems_to_test`!")
    define_system_func = None

Připraven ke spuštění experimentu pro systém: '1D Nelineární (kubický)'


In [5]:
# ==============================================================================
# KROK 4b: SPUŠTĚNÍ JEDNOHO KOMPLETNÍHO EXPERIMENTU
# ==============================================================================

# Zkontrolujeme, zda byl systém v předchozí buňce správně vybrán
if 'define_system_func' in locals() and define_system_func is not None:
    
    # Tento blok se provede pouze pro systém nakonfigurovaný v buňce 4a
    
    print("\n" + "#"*100)
    print(f"# ZPRACOVÁVÁM SYSTÉM: {system_name_to_run.upper()}")
    print("#"*100)

    # 1. Definice a inicializace aktuálního systému
    sys_true, sys_model = define_system_func()

    # 2. Generování dat pro tento specifický systém
    print("\n--- Generuji data pro aktuální systém ---")
    x_train, y_train = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TRAIN_TRAJ"], seq_len=DATA_CONFIG["TRAIN_SEQ_LEN"])
    x_val, y_val = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_VALID_TRAJ"], seq_len=DATA_CONFIG["VALID_SEQ_LEN"])
    x_test, y_test = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TEST_TRAJ"], seq_len=DATA_CONFIG["TEST_SEQ_LEN"])
    
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    print("--- Generování dat dokončeno. ---")

    # 3. Spuštění tréninku všech NN modelů pro tento systém
    # Tato funkce vrátí slovník s natrénovanými modely.
    trained_models = run_training_for_system(train_loader, val_loader, sys_model, device)
    
    # 4. Spuštění finální evaluace s natrénovanými modely
    # Tato funkce provede kompletní srovnání a vytiskne finální tabulku.
    run_evaluation_for_system(trained_models, test_loader, sys_true, sys_model, device)

    print("\n\n" + "#"*100)
    print(f"# EXPERIMENT PRO SYSTÉM '{system_name_to_run.upper()}' DOKONČEN")
    print("#"*100)

else:
    print("\nCHYBA: Spuštění bylo přeskočeno.")
    print("Prosím, nejprve spusťte buňku 4a a ujistěte se, že `system_name_to_run` je správně nastaven.")


####################################################################################################
# ZPRACOVÁVÁM SYSTÉM: 1D NELINEÁRNÍ (KUBICKÝ)
####################################################################################################

--- Definuji Systém 1: 1D Nelineární (kubický) ---

--- Generuji data pro aktuální systém ---
--- Generování dat dokončeno. ---

ZAČÍNÁM TRÉNINKOVOU FÁZI PRO AKTUÁLNÍ SYSTÉM

----------------------------------------
--- Trénuji KalmanNet with Known R (KNetR) ---
----------------------------------------


/home/luky/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


INFO: Detekováno, že model vrací kovarianci: True
Epoch [5/150], Train Loss: 0.322528, Val Loss: 0.268648, Avg Cov Trace: 0.009598
Epoch [10/150], Train Loss: 0.171118, Val Loss: 0.138656, Avg Cov Trace: 0.150973
Epoch [15/150], Train Loss: 0.152185, Val Loss: 0.133493, Avg Cov Trace: 0.220755
Epoch [20/150], Train Loss: 0.144953, Val Loss: 0.133006, Avg Cov Trace: 0.241236
Epoch [25/150], Train Loss: 0.141007, Val Loss: 0.131582, Avg Cov Trace: 0.242071
Epoch [30/150], Train Loss: 0.138058, Val Loss: 0.130916, Avg Cov Trace: 0.241965
Epoch [35/150], Train Loss: 0.135941, Val Loss: 0.130761, Avg Cov Trace: 0.241670
Epoch [40/150], Train Loss: 0.134909, Val Loss: 0.130885, Avg Cov Trace: 0.240464
Epoch [45/150], Train Loss: 0.133997, Val Loss: 0.130965, Avg Cov Trace: 0.242163
Epoch [50/150], Train Loss: 0.133421, Val Loss: 0.131028, Avg Cov Trace: 0.242418
Epoch [55/150], Train Loss: 0.132745, Val Loss: 0.131031, Avg Cov Trace: 0.242191
Epoch [60/150], Train Loss: 0.133020, Val Loss: 0

IndexError: tuple index out of range

In [7]:
# ==============================================================================
# BUŇKA 5: FINÁLNÍ EVALUACE NATRÉNOVANÝCH MODELŮ
# ==============================================================================

# --- Zde jen zkontrolujte, že máte správně přiřazené natrénované modely ---
# Tento blok je jen pro přehlednost, nemusíte ho měnit, pokud máte proměnné správně.
try:
    trained_model_bkn = trained_models['bkn']
    # trained_model_classic = trained_models['classic_knet']
    trained_model_knetR = trained_models['knetR']
    print("INFO: Natrénované modely nalezeny.")
except (NameError, KeyError):
    print("VAROVÁNÍ: Proměnná 'trained_models' nenalezena. Ujistěte se, že máte modely natrénované a uložené v proměnných `trained_model_bkn`, `trained_model_classic` a `trained_model_knetR`.")


# --- Konfigurace testu ---
TEST_SEQ_LEN = 300
NUM_TEST_TRAJ = 20
J_SAMPLES_TEST = 25

# --- Příprava dat ---
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# --- Inicializace filtrů ---
# DŮLEŽITÉ: Ujistěte se, že vaše třídy EKF a UKF obsahují opravu s .flatten()
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
print("Model-based filtry (EKF, UKF) inicializovány.")

# --- Seznamy pro ukládání výsledků ---
all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

# Přepnutí NN modelů do evaluačního režimu
trained_model_bkn.eval()
# trained_model_classic.eval()
trained_model_knetR.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # A. Bayesian KalmanNet
        ensemble_trajectories = []
        for j in range(J_SAMPLES_TEST):
            trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
            current_x_hats = []
            for t in range(1, TEST_SEQ_LEN):
                x_filtered_t, _ = trained_model_bkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
                current_x_hats.append(x_filtered_t)
            ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
        ensemble = torch.stack(ensemble_trajectories, dim=0)
        predictions_bkn = ensemble.mean(dim=0)
        diff = ensemble - predictions_bkn.unsqueeze(0)
        covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
        full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        full_P_hat_bkn = torch.cat([sys_model.P0.unsqueeze(0), covariances_bkn], dim=0)

        # B. Klasický KNet
        # trained_model_classic.reset(batch_size=1, initial_state=initial_state)
        # classic_knet_preds = []
        # for t in range(1, TEST_SEQ_LEN):
        #     x_filtered_t = trained_model_classic.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #     classic_knet_preds.append(x_filtered_t)
        # full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        
        # C. KNet with Known R
        trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        knetR_preds_x, knetR_preds_P = [], []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
            knetR_preds_x.append(x_filtered_t)
            knetR_preds_P.append(P_filtered_t)
        full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        full_P_hat_knetR = torch.cat([sys_model.P0.unsqueeze(0), torch.stack(knetR_preds_P, dim=0).squeeze(1)], dim=0)

        # D. EKF
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ekf_m = torch.cat([sys_model.Ex0.reshape(1, -1), ekf_m_res['x_filtered']], dim=0)
        full_P_hat_ekf_m = torch.cat([sys_model.P0.unsqueeze(0), ekf_m_res['P_filtered']], dim=0)
        
        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ekf_i = torch.cat([sys_true.Ex0.reshape(1, -1), ekf_i_res['x_filtered']], dim=0)
        full_P_hat_ekf_i = torch.cat([sys_true.P0.unsqueeze(0), ekf_i_res['P_filtered']], dim=0)

        # E. UKF
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ukf_m = torch.cat([sys_model.Ex0.reshape(1, -1), ukf_m_res['x_filtered']], dim=0)
        full_P_hat_ukf_m = torch.cat([sys_model.P0.unsqueeze(0), ukf_m_res['P_filtered']], dim=0)
        
        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ukf_i = torch.cat([sys_true.Ex0.reshape(1, -1), ukf_i_res['x_filtered']], dim=0)
        full_P_hat_ukf_i = torch.cat([sys_true.P0.unsqueeze(0), ukf_i_res['P_filtered']], dim=0)
        
        # F. Uložení výsledků
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        # all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

    # --- Finální výpočet a výpis metrik ---
    mse_bkn, anees_bkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
    mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
    mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
    
    print("\nPočítám finální metriky pro jednotlivé trajektorie...")
    with torch.no_grad():
        for i in range(NUM_TEST_TRAJ):
            x_true = all_x_true_cpu[i]
            def get_metrics(x_hat, P_hat):
                mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
                anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
                return mse, anees

            mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        #  mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
            mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
            mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
            mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
            mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
            mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)

    def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
    state_dim_for_nees = all_x_true_cpu[0].shape[1]

    print("\n" + "="*80)
    print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
    print("="*80)
    print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
    print("-" * 80)
    print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
    print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
    # print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
    print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
    print("-" * 80)
    print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
    print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
    print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
    print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
    print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
    print("="*80)

INFO: Natrénované modely nalezeny.

Generuji 20 testovacích trajektorií o délce 300...
Generování dat dokončeno.
Model-based filtry (EKF, UKF) inicializovány.

Vyhodnocuji modely na 20 testovacích trajektoriích...


IndexError: tuple index out of range

In [ ]:
# ==============================================================================
# BUŇKA PRO RYCHLÝ TEST FUNKČNOSTI EKF a UKF NA 1D SYSTÉMU
# ==============================================================================
# Cíl: Ověřit, že filtry běží bez chyb a produkují správné výstupy.

import torch
import sys
import os
from math import pi

# --- KROK 1: Importy a základní nastavení ---
# Předpokládá se, že cesta k projektu je již nastavena.
# Pokud ne, odkomentujte následující řádky:
# notebook_path = os.getcwd()
# project_root = os.path.dirname(notebook_path)
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

import Systems
import Filters

# Použijeme stejnou definici třídy DynamicSystem, kterou jste poskytl
from Systems import DynamicSystem 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

# --- KROK 2: Definice testovacího systému ---
# Přesná kopie vašeho 1D nelineárního systému
def define_system_1d_nonlinear():
    state_dim, obs_dim = 1, 1
    
    # Skutečná dynamika
    f_true = lambda x: 0.9 * x - 0.05 * x**3 
    h_true = lambda x: 0.5 * x
    Q_true = torch.tensor([[0.1]])
    R_true = torch.tensor([[0.1]])
    Ex0_true = torch.tensor([[1.0]])
    P0_true = torch.tensor([[0.5]])
    sys_true = DynamicSystem(state_dim, obs_dim, f=f_true, h=h_true, Q=Q_true, R=R_true, Ex0=Ex0_true, P0=P0_true, device=device)

    # Nepřesný model
    f_model = lambda x: 0.9 * x 
    Q_model = torch.tensor([[0.01]])
    R_model = torch.tensor([[0.2]])
    Ex0_model = torch.tensor([[0.5]])
    P0_model = torch.tensor([[0.5]])
    sys_model = DynamicSystem(state_dim, obs_dim, f=f_model, h=h_true, Q=Q_model, R=R_model, Ex0=Ex0_model, P0=P0_model, device=device)
    
    return sys_true, sys_model

# --- KROK 3: Generování jedné testovací trajektorie ---
# Nepotřebujeme DataLoader, vygenerujeme si data ručně pro jednoduchost
TEST_SEQ_LEN = 50
print(f"\nDefinuji systém a generuji jednu trajektorii o délce {TEST_SEQ_LEN}...")
sys_true, sys_model = define_system_1d_nonlinear()

x_true_list = []
y_test_list = []
x_current = sys_true.get_initial_state()

for _ in range(TEST_SEQ_LEN):
    x_true_list.append(x_current)
    y_current = sys_true.measure(x_current)
    y_test_list.append(y_current)
    x_current = sys_true.step(x_current)

x_true_tensor = torch.stack(x_true_list, dim=0).to(device)
y_test_tensor = torch.stack(y_test_list, dim=0).to(device)
print("--- Testovací data vygenerována. ---")
print(f"Tvar tenzoru měření y_test_tensor: {y_test_tensor.shape}")

# --- KROK 4: Spuštění filtrů a ověření výstupů ---
print("\n" + "="*80)
print("ZAČÍNÁM TESTOVÁNÍ FILTRŮ...")
print("="*80)

# Inicializace všech filtrů
try:
    ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
    ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
    ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
    ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
    print("Všechny filtry úspěšně inicializovány.\n")
except Exception as e:
    print(f"CHYBA při inicializaci filtrů: {e}")

# Test EKF (ideální)
try:
    print("--- Testuji EKF (Ideální model) ---")
    results = ekf_ideal.process_sequence(y_test_tensor, Ex0=sys_true.Ex0, P0=sys_true.P0)
    print(">>> ÚSPĚCH <<<")
    print(f"    Výstup 'x_filtered': {results['x_filtered'].shape}")
    print(f"    Výstup 'P_filtered': {results['P_filtered'].shape}")
except Exception as e:
    print(f">>> SELHÁNÍ <<< Chyba: {e}")

# Test EKF (nepřesný)
try:
    print("\n--- Testuji EKF (Nepřesný model) ---")
    results = ekf_mismatched.process_sequence(y_test_tensor, Ex0=sys_model.Ex0, P0=sys_model.P0)
    print(">>> ÚSPĚCH <<<")
    print(f"    Výstup 'x_filtered': {results['x_filtered'].shape}")
    print(f"    Výstup 'P_filtered': {results['P_filtered'].shape}")
except Exception as e:
    print(f">>> SELHÁNÍ <<< Chyba: {e}")

# Test UKF (ideální)
try:
    print("\n--- Testuji UKF (Ideální model) ---")
    results = ukf_ideal.process_sequence(y_test_tensor, Ex0=sys_true.Ex0, P0=sys_true.P0)
    print(">>> ÚSPĚCH <<<")
    print(f"    Výstup 'x_filtered': {results['x_filtered'].shape}")
    print(f"    Výstup 'P_filtered': {results['P_filtered'].shape}")
except Exception as e:
    print(f">>> SELHÁNÍ <<< Chyba: {e}")
    
# Test UKF (nepřesný)
try:
    print("\n--- Testuji UKF (Nepřesný model) ---")
    results = ukf_mismatched.process_sequence(y_test_tensor, Ex0=sys_model.Ex0, P0=sys_model.P0)
    print(">>> ÚSPĚCH <<<")
    print(f"    Výstup 'x_filtered': {results['x_filtered'].shape}")
    print(f"    Výstup 'P_filtered': {results['P_filtered'].shape}")
except Exception as e:
    print(f">>> SELHÁNÍ <<< Chyba: {e}")

print("\n" + "="*80)
print("TESTOVÁNÍ DOKONČENO.")
print("="*80)

In [ ]:
# system_name_to_run = "1D Nelineární (kubický)"
system_name_to_run = "2D Syntetický (sinusový)"
# system_name_to_run = "2D Lineární (easy)"

# Zkontrolujeme, zda byl systém v předchozí buňce správně vybrán
if 'define_system_func' in locals() and define_system_func is not None:
    
    # Tento blok se provede pouze pro systém nakonfigurovaný v buňce 4a
    
    print("\n" + "#"*100)
    print(f"# ZPRACOVÁVÁM SYSTÉM: {system_name_to_run.upper()}")
    print("#"*100)

    # 1. Definice a inicializace aktuálního systému
    sys_true, sys_model = define_system_func()

    # 2. Generování dat pro tento specifický systém
    print("\n--- Generuji data pro aktuální systém ---")
    x_train, y_train = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TRAIN_TRAJ"], seq_len=DATA_CONFIG["TRAIN_SEQ_LEN"])
    x_val, y_val = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_VALID_TRAJ"], seq_len=DATA_CONFIG["VALID_SEQ_LEN"])
    x_test, y_test = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TEST_TRAJ"], seq_len=DATA_CONFIG["TEST_SEQ_LEN"])
    
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    print("--- Generování dat dokončeno. ---")

    # 3. Spuštění tréninku všech NN modelů pro tento systém
    # Tato funkce vrátí slovník s natrénovanými modely.
    trained_models = run_training_for_system(train_loader, val_loader, sys_model, device)
    
    # 4. Spuštění finální evaluace s natrénovanými modely
    # Tato funkce provede kompletní srovnání a vytiskne finální tabulku.
    run_evaluation_for_system(trained_models, test_loader, sys_true, sys_model, device)

    print("\n\n" + "#"*100)
    print(f"# EXPERIMENT PRO SYSTÉM '{system_name_to_run.upper()}' DOKONČEN")
    print("#"*100)

else:
    print("\nCHYBA: Spuštění bylo přeskočeno.")
    print("Prosím, nejprve spusťte buňku 4a a ujistěte se, že `system_name_to_run` je správně nastaven.")

In [ ]:
system_name_to_run = "2D Lineární (easy)"

# Zkontrolujeme, zda byl systém v předchozí buňce správně vybrán
if 'define_system_func' in locals() and define_system_func is not None:
    
    # Tento blok se provede pouze pro systém nakonfigurovaný v buňce 4a
    
    print("\n" + "#"*100)
    print(f"# ZPRACOVÁVÁM SYSTÉM: {system_name_to_run.upper()}")
    print("#"*100)

    # 1. Definice a inicializace aktuálního systému
    sys_true, sys_model = define_system_func()

    # 2. Generování dat pro tento specifický systém
    print("\n--- Generuji data pro aktuální systém ---")
    x_train, y_train = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TRAIN_TRAJ"], seq_len=DATA_CONFIG["TRAIN_SEQ_LEN"])
    x_val, y_val = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_VALID_TRAJ"], seq_len=DATA_CONFIG["VALID_SEQ_LEN"])
    x_test, y_test = utils.generate_data(sys_true, num_trajectories=DATA_CONFIG["NUM_TEST_TRAJ"], seq_len=DATA_CONFIG["TEST_SEQ_LEN"])
    
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=DATA_CONFIG["BATCH_SIZE"], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    print("--- Generování dat dokončeno. ---")

    # 3. Spuštění tréninku všech NN modelů pro tento systém
    # Tato funkce vrátí slovník s natrénovanými modely.
    trained_models = run_training_for_system(train_loader, val_loader, sys_model, device)
    
    # 4. Spuštění finální evaluace s natrénovanými modely
    # Tato funkce provede kompletní srovnání a vytiskne finální tabulku.
    run_evaluation_for_system(trained_models, test_loader, sys_true, sys_model, device)

    print("\n\n" + "#"*100)
    print(f"# EXPERIMENT PRO SYSTÉM '{system_name_to_run.upper()}' DOKONČEN")
    print("#"*100)

else:
    print("\nCHYBA: Spuštění bylo přeskočeno.")
    print("Prosím, nejprve spusťte buňku 4a a ujistěte se, že `system_name_to_run` je správně nastaven.")